# pyboj クイックスタート

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obichan117/pyboj/blob/main/examples/quickstart.ipynb)

[日本銀行 時系列統計データ検索サイト](https://www.stat-search.boj.or.jp/) の Python クライアントです。

すべてのパラメータは型付き Enum で指定 — マジックストリング不要。
メタデータを自動取得し、条件に合うシリーズをフィルタリングして、
日付・数値のパース済みドメインオブジェクトを返します。

In [ ]:
!pip install -q pyboj[plot]

In [ ]:
import matplotlib.pyplot as plt
from pyboj import (
    BOJ,
    BopAccount,
    Currency,
    Database,
    FofSector,
    Frequency,
    IndexType,
    IndustrySector,
    Lang,
    MonetaryComponent,
    RateCategory,
    TankanIndustry,
    TankanSize,
    plot_series,
)

boj = BOJ()  # デフォルトで日本語ラベル (lang=Lang.JP)

## 1. 為替レート

ドル円スポットレート。通貨ペアと頻度で絞り込み。

In [ ]:
rates = boj.exchange_rates(
    currency=Currency.USD_JPY,
    frequency=Frequency.D,
    start_date="202401",
)

r = rates[0]
print(f"{r.name_jp} | {len(r.dates)} 件")

# .plot() で日本語ラベルが自動設定される
r.plot()
plt.show()

### 組み込みプロット機能

すべての Series オブジェクトに `.plot()` メソッドがあります。
`BOJ()` のデフォルト言語に従い、日本語ラベルが自動設定されます。
`lang=Lang.EN` で英語ラベルに切り替え可能。

In [ ]:
# pandas DataFrame への変換
df = r.to_dataframe()
print(df.head())

# 英語ラベルで表示したい場合
r.plot(lang=Lang.EN)
plt.show()

## 2. 金利

コールレート、レポレート、政策金利。カテゴリと担保の有無で絞り込み。

In [ ]:
ir = boj.interest_rates(
    category=RateCategory.CALL_RATE,
    frequency=Frequency.D,
    start_date="202401",
)

# 複数系列を plot_series で重ね描き
if ir:
    plot_series(*ir[:4])
    plt.show()

## 3. 短観（TANKAN）

約1万社を対象とした四半期ごとの景気判断調査。DI > 0 は「良い」と回答した企業が多いことを意味します。

In [ ]:
tankan = boj.tankan(
    industry=TankanIndustry.MANUFACTURING,
    size=TankanSize.LARGE,
    start_date="201501",
)

# 業況判断 DI を探す
di_series = [t for t in tankan if "business conditions" in (t.name or "").lower()]
if di_series:
    s = di_series[0]
    print(f"{s.name_jp} | {len(s.dates)} 件")
    ax = s.plot()
    ax.axhline(y=0, color="black", linewidth=0.8, linestyle="--")
    plt.show()

## 4. 物価指数

企業物価指数 — 消費者物価の先行指標として注目されます。

In [ ]:
indices = boj.price_indices(
    index_type=IndexType.PRODUCER,
    start_date="202001",
)

if indices:
    s = indices[0]
    print(f"{s.name_jp} | 基準年={s.base_year} | {len(s.dates)} 件")
    s.plot()
    plt.show()

## 5. 国際収支

経常収支 — 貿易・サービス・所得の純フロー。

In [ ]:
bop = boj.balance_of_payments(
    account=BopAccount.CURRENT,
    frequency=Frequency.M,
    start_date="202001",
)

if bop:
    s = bop[0]
    print(f"{s.name_jp} | {s.unit_jp} | {len(s.dates)} 件")
    ax = s.plot()
    ax.axhline(y=0, color="black", linewidth=0.8, linestyle="--")
    plt.show()

## 6. マネタリーベース

日銀が供給する通貨の総量。量的緩和の規模を示します。

In [ ]:
money = boj.money_deposits(
    component=MonetaryComponent.TOTAL,
    db=Database.MONETARY_BASE,
    start_date="200501",
)

if money:
    s = money[0]
    print(f"{s.name_jp} | {s.unit_jp} | {len(s.dates)} 件")
    s.plot()
    plt.show()

## 7. 業種別貸出

業種別の銀行貸出残高。

In [ ]:
loans = boj.loans(
    sector=IndustrySector.MANUFACTURING,
    start_date="202001",
)

if loans:
    s = loans[0]
    print(f"{s.name_jp} | {s.unit_jp} | {len(s.dates)} 件")
    s.plot()
    plt.show()

## 8. 資金循環

家計の金融資産 — 日本の家計が保有する金融資産の総額。

In [ ]:
fof = boj.flow_of_funds(
    sector=FofSector.HOUSEHOLDS,
    start_date="200501",
)

if fof:
    s = fof[0]
    print(f"{s.name_jp} | {s.unit_jp} | {len(s.dates)} 件")
    s.plot()
    plt.show()

## その他のカテゴリ

`BOJ` は全13カテゴリ（43データベース）をカバーしています。

In [ ]:
from pyboj import (
    InstitutionType,
    MarketSegment,
    OperationType,
    StatCategory,
)

# 金融市場
fm = boj.financial_markets(segment=MarketSegment.GOVT_BONDS, db=Database.GOVT_BOND_TRADING)
print(f"金融市場: {len(fm)} 系列")

# 貸借対照表
bs = boj.balance_sheets(institution_type=InstitutionType.BOJ)
print(f"貸借対照表: {len(bs)} 系列")

# 日銀オペ
ops = boj.boj_operations(operation_type=OperationType.JGB_OPERATIONS)
print(f"日銀オペ: {len(ops)} 系列")

# 財政
pf = boj.public_finance()
print(f"財政: {len(pf)} 系列")

# 国際統計
intl = boj.international(stat_category=StatCategory.DERIVATIVES, db=Database.DERIVATIVES_MARKET)
print(f"国際統計: {len(intl)} 系列")

## データ探索: 階層ツリーとキーワード検索

どんなデータがあるか分からない場合は、`layer_tree()` で階層を閲覧するか、
`search()` でキーワード検索できます。

In [ ]:
# 為替レートの階層ツリーを表示
tree = boj.layer_tree(Database.EXCHANGE_RATES)
print(f"トップレベル: {len(tree.children)} カテゴリ\n")
for child in tree.children:
    print(f"  {child.name} ({len(child.series_codes)} 系列, {len(child.children)} サブ)")

In [ ]:
# キーワードで検索
results = boj.search(Database.EXCHANGE_RATES, "ドル")
print(f"「ドル」に一致: {len(results)} 系列\n")
for rec in results[:5]:
    print(f"  {rec.SERIES_CODE} | {rec.NAME_OF_TIME_SERIES_J}")

boj.close()